# fftboost: quick start

Minimal, visual introduction. Configure, run CV, inspect acceptance (J-Beats), and view a simple FFT snapshot.

In [ ]:
# Install FFTBoost
!pip install -q git+https://github.com/pinballsurgeon/fftboost.git
!pip install -q matplotlib

import json, time, math
import numpy as np
import matplotlib.pyplot as plt

try:
    from fftboost.api import FFTBoost, load_config_from_yaml
except Exception as e:
    raise RuntimeError("Install fftboost first: !pip install -q git+https://github.com/pinballsurgeon/fftboost.git") from e

np.set_printoptions(suppress=True, linewidth=120)

## configure
Fast settings for notebook runs (short duration, few folds).

In [ ]:
%%writefile demo_config.yaml
seed: 1337
fs: 1000
duration_s: 16
window_s: 0.4
hop_s: 0.2
cv_folds: 3

use_wavelets: true
wavelet_family: "db4"
wavelet_level: 4
use_hilbert_phase: true

coherence_subbands:
  - [1, 40]
  - [40, 100]

target:
  ehi_weights: { thd: 0.45, ipr: 0.55 }

fftboost:
  atoms: 6
  lambda_hf: 0.10
  lambda_coh: 3.0
  min_sep_bins: 3
  ridge_alpha: 0.1

## run cross-validation
Outputs telemetry including the J-Beats acceptance gate (ΔR² CI > 0).

In [ ]:
config = load_config_from_yaml("demo_config.yaml")
m = FFTBoost(config)
t0 = time.perf_counter()
m.run_evaluation_with_generator()
t1 = time.perf_counter()
print(f"evaluation wall time: {(t1-t0):.3f}s")

# J-Beats telemetry (public API)
tele = m.get_j_beats_telemetry()

# robust JSON print (numpy-safe)
class _NumpyEncoder(json.JSONEncoder):
    def default(self, o):
        import numpy as _np
        if isinstance(o, (_np.integer, )): return int(o)
        if isinstance(o, (_np.floating, )): return float(o)
        if isinstance(o, (_np.ndarray, )): return o.tolist()
        if isinstance(o, (_np.bool_, )): return bool(o)
        return super().default(o)

print(json.dumps({k: tele.get(k, None) for k in ["j_beats_pass","mean_delta","ci_low","ci_high"]}, indent=2, cls=_NumpyEncoder))

## visualize acceptance
Mean ΔR² with 95% CI. If fold deltas are available, plot them as well.

In [ ]:
mean_d = tele.get("mean_delta", 0.0)
ci_lo  = tele.get("ci_low", mean_d)
ci_hi  = tele.get("ci_high", mean_d)

plt.figure(figsize=(4,3))
mid = (ci_lo+ci_hi)/2
err = [[mid-ci_lo],[ci_hi-mid]]
plt.errorbar([0], [mean_d], yerr=err, fmt='o', capsize=4)
plt.axhline(0, linestyle='--')
plt.xticks([0],["ΔR² (mean)"])
plt.ylabel("delta R² vs baseline")
plt.title("J-Beats summary")
plt.tight_layout()
plt.show()

# Optional per-fold bar if present (key names may vary)
folds = tele.get("folds") or tele.get("per_fold") or []
if folds:
    deltas = []
    for f in folds:
        # accept common field names
        d = f.get('delta') or f.get('dFFT') or f.get('delta_r2')
        if d is not None: deltas.append(d)
    if deltas:
        plt.figure(figsize=(5,3))
        x = np.arange(len(deltas))
        plt.bar(x, deltas)
        plt.axhline(0, linestyle='--')
        plt.xlabel("fold")
        plt.ylabel("ΔR²")
        plt.title("per-fold ΔR²")
        plt.tight_layout()
        plt.show()

## fft snapshot
Single window, magnitude FFT. This illustrates the sparse-in-frequency use case `fftboost` targets (fundamental, harmonics, interharmonics).

In [ ]:
# simple synthetic window for illustration (independent of library internals)
Fs = 1000
T = 0.4
N = int(Fs*T)
t = np.arange(N)/Fs
rng = np.random.default_rng(123)
x = 1.0*np.sin(2*np.pi*50*t) + 0.4*np.sin(2*np.pi*100*t) + 0.2*np.sin(2*np.pi*125*t) + 0.2*rng.standard_normal(N)
X = np.abs(np.fft.rfft(x*np.hanning(N)))
f = np.fft.rfftfreq(N, 1/Fs)

plt.figure(figsize=(6,3))
plt.plot(f, X)
plt.xlim(0, 250)
plt.xlabel("frequency (Hz)")
plt.ylabel("|FFT|")
plt.title("one window: FFT magnitude")
plt.tight_layout()
plt.show()

## latency snapshot
Rough timing for a small batch of FFTs + basic scaling to reflect the main inference path cost profile.

In [ ]:
def _latency_probe(n_windows=128, fs=1000, win_s=0.4):
    N = int(fs*win_s)
    rng = np.random.default_rng(7)
    W = np.hanning(N)
    batch = rng.standard_normal((n_windows, N))
    t0 = time.perf_counter()
    Xm = np.abs(np.fft.rfft(batch*W, axis=1))
    mu = Xm.mean(axis=0); sd = Xm.std(axis=0) + 1e-9
    Xs = (Xm - mu)/sd
    # trivial linear head to mimic final projection cost
    w = rng.standard_normal(Xs.shape[1])
    y = Xs @ w
    t1 = time.perf_counter()
    return ((t1-t0)*1000)/n_windows

ms = _latency_probe()
print(f"approx per-window latency: {ms:.3f} ms (FFT + scale + dot)")

## cli
The same run via CLI (uncomment in environments where the console script is on PATH).

In [ ]:
!fftboost --config demo_config.yaml